Welcome to a Dataquest guided project! If you haven't gone through a guided project yet, we recommend starting with the first one in the command line course.

In this project, you'll be working with data from the S&P500 Index. The S&P500 is a stock market index. Before we get into what an index is, we'll need to get into the basics of the stock market.

Some companies are publicly traded, which means that anyone can buy and sell their shares on the open market. A share entitles the owner to some control over the direction of the company, and to some percentage (or share) of the earnings of the company. When you buy or sell shares, it's common to say that you're trading a stock.

The price of a share is based mainly on supply and demand for a given stock. For example, Apple stock has a price of 120 dollars per share as of December 2015 -- http://www.nasdaq.com/symbol/aapl. A stock that is in less demand, like Ford Motor Company, has a lower price -- http://finance.yahoo.com/q?s=F. Stock price is also influenced by other factors, including the number of shares a company has issued.

Stocks are traded daily, and the price can rise or fall from the beginning of a trading day to the end based on demand. Stocks that are in more in demand, such as Apple, are traded more often than stocks of smaller companies.

Indexes aggregate the prices of multiple stocks together, and allow you to see how the market as a whole is performing. For example, the Dow Jones Industrial Average aggregates the stock prices of 30 large American companies together. The S&P500 Index aggregates the stock prices of 500 large companies. When an index fund goes up or down, you can say that the underlying market or sector it represents is also going up or down. For example, if the Dow Jones Industrial Average price goes down one day, you can say that American stocks overall went down (ie, most American stocks went down in price).

You'll be using historical data on the price of the S&P500 Index to make predictions about future prices. Predicting whether an index will go up or down will help us forecast how the stock market as a whole will perform. Since stocks tend to correlate with how well the economy as a whole is performing, it can also help us make economic forecasts.

There are also thousands of traders who make money by buying and selling Exchange Traded Funds. ETFs allow you to buy and sell indexes like stocks. This means that you could "buy" the S&P500 Index ETF when the price is low, and sell when it's high to make a profit. Creating a predictive model could allow traders to make money on the stock market.

Note: You shouldn't make trades with any models developed in this mission. Trading stocks has risks, and nothing in this mission constitutes stock trading advice.

In this mission, you'll be working with a csv file containing index prices. Each row in the file contains a daily record of the price of the S&P500 Index from 1950 to 2015. The dataset is stored in sphist.csv.

The columns of the dataset are:

- Date -- The date of the record.
- Open -- The opening price of the day (when trading starts).
- High -- The highest trade price during the day.
- Low -- The lowest trade price during the day.
- Close -- The closing price for the day (when trading is finished).
- Volume -- The number of shares traded.
- Adj Close -- The daily closing price, adjusted retroactively to include any corporate actions. Read more here.

You'll be using this dataset to develop a predictive model. You'll train the model with data from 1950-2012, and try to make predictions from 2013-2015.

You'll need to read the data into Python, do some processing to set the right column types, and then sort the dataframe. You can do this in the predict.py script.

<b>Instructions</b><br>
Here are the steps you'll need to take, at a high level:

- Read the data into a Pandas DataFrame. You can use the read_csv Pandas function for this.
- Convert the Date column to a Pandas date type. This will allow you to do date comparisons with the column.
    - You can perform this conversion with the to_datetime function in Pandas.
    - Once you convert the column, you can perform comparisons with df["Date"] > datetime(year=2015, month=4, day=1). This will generate a Boolean series that tells you if each item in the Date column is after 2015-04-01. You'll have to import the datetime module from the datetime library first with from datetime import datetime.
- Sort the dataframe on the Date column. It's currently in descending order, but we'll want it to be in ascending order for some of the next steps. You can use the DataFrame.sort_values() method on data frames for this.

Make sure to run the predict.py script using python predict.py as you work through the steps.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv('sphist.csv')

# Convert Date column to datetime

df['Date'] = pd.to_datetime(df['Date'])
df.dtypes

# Sort df by Date in ascending order

df.sort_values(by='Date', ascending=True, inplace=True)

In [3]:
df.head()

,Date,Open,High,Low,Close,Volume,Adj Close
16589,1950-01-03,16.66,16.66,16.66,16.66,1260000.0,16.66
16588,1950-01-04,16.85,16.85,16.85,16.85,1890000.0,16.85
16587,1950-01-05,16.93,16.93,16.93,16.93,2550000.0,16.93
16586,1950-01-06,16.98,16.98,16.98,16.98,2010000.0,16.98
16585,1950-01-09,17.08,17.08,17.08,17.08,2520000.0,17.08


Datasets taken from the stock market need to be handled differently than datasets from other sectors when it comes time to make predictions. In a normal machine learning exercise, we treat each row as independent. Stock market data is sequential, and each observation comes a day after the previous observation. Thus, the observations are not all independent, and you can't treat them as such.

This means you have to be extra careful to not inject "future" knowledge into past rows when you do training and prediction. Injecting future knowledge will make our model look good when you're training and testing it, but will make it fail in the real world. This is how many algorithmic traders lose money.

The time series nature of the data means that you can generate indicators to make our model more accurate. For instance, you can create a new column that contains the average price of the last 10 trades for each row. This will incorporate information from multiple prior rows into one, and will make predictions much more accurate.

When you do this, you have to be careful not to use the current row in the values you average. You want to teach the model how to predict the current price from historical prices. If you include the current price in the prices you average, it will be equivalent to handing the answers to the model upfront, and will make it impossible to use in the "real world", where you don't know the price upfront.

Here are some indicators that are interesting to generate for each row:

- The average price from the past 5 days.
- The average price for the past 30 days.
- The average price for the past 365 days.
- The ratio between the average price for the past 5 days, and the average price for the past 365 days.
- The standard deviation of the price over the past 5 days.
- The standard deviation of the price over the past 365 days.
- The ratio between the standard deviation for the past 5 days, and the standard deviation for the past 365 days.

"Days" means "trading days" -- so if you're computing the average of the past 5 days, it should be the 5 most recent dates before the current one. Assume that "price" means the Close column. Always be careful not to include the current price in these indicators! You're predicting the next day price, so our indicators are designed to predict the current price from the previous prices.

Some of these indicators require a year of historical data to compute. Our first day of data falls on 1950-01-03, so the first day you can start computing indicators on is 1951-01-03.

To compute indicators, you'll need to loop through each day from 1951-01-03 to 2015-12-07 (the last day you have prices for). For instance, if we were computing the average price from the past 5 days, we'd start at 1951-01-03, get the prices for each day from 1950-12-26 to 1951-01-02, and find the average. The reason why we start on the 26th, and take more than 5 calendar days into account is because the stock market is shutdown on certain holidays. Since we're looking at the past 5 trading days, we need to look at more than 5 calendar days to find them. Here's a diagram showing how we average 5 days to get the average closing price for 1951-01-03:

<img src='stocks/fig1.jpg', height=371, width=348>

You'd then move to 1951-01-04, and find the average price from 1950-12-30 to 1951-01-03. Here's a diagram showing how we might compute the average here:

<img src='stocks/fig2.jpg', height=349, width=384>

We'd keep repeating this process to compute all of the averages. Note how when we compute the average of the past 5 days for 1951-01-04, we don't include 1951-01-04 in that average. It's critical not to do this, or our model won't work in the "real world".

Here's a table of how the first 10 dates would look if we computed the 5 day average closing price. Close is the closing price for that day, and day_5 is the average of the past 5 trading closing prices at each row:

<img src='stocks/fig3.jpg', height=385, width=428>

<b>Instructions</b><br>
Pick 3 indicators to compute, and generate a different column for each one.

There are a few different ways to do this:

- You can use a for loop along with the iterrows method to loop over the rows in the DataFrame and compute the indicators. This is the recommended way, as it's a bit simpler to understand what's happening. Since you'll be looping over all of the rows, for any date that comes before there is enough historical data to compute an indicator, just fill in 0.
- Pandas has some time series tools that can help, including the rolling_mean function, which will do most of the hard computation for you. Set the window equal to the number of trading days in the past you want to use to compute the indicators. This will add in NaN values for any row where there aren't enough historical trading days to do the computation. Note: There is a giant caveat here, which is that the rolling mean will use the current day's price. You'll need to reindex the resulting series to shift all the values "forward" one day. For example, the rolling mean calculated for 1950-01-03 will need to be assigned to 1950-01-04, and so on. You can use the shift method on Dataframes to do this.

In [4]:
# First method: 5 day average

close_values = []
means = []
stds = []
indices = []

for index, row in df.iterrows():
    close_values.append(row['Close'])
    indices.append(index)
    if len(close_values) > 5:
        last_five = close_values[-6:-1]
        means.append(np.mean(last_five))
        stds.append(np.std(last_five, ddof=1))
    else:
        means.append(0)
        stds.append(0)

df['day_5'] = pd.Series(means, index=indices)
df['std_5'] = pd.Series(stds, index=indices)

In [5]:
# Check to make sure that answers match the ones in the explanation above

df[(datetime(year=1951, month=1, day=2) < df['Date']) & (datetime(year=1951, month=1, day=17) > df['Date'])]

,Date,Open,High,Low,Close,Volume,Adj Close,day_5,std_5
16339,1951-01-03,20.690001,20.690001,20.690001,20.690001,3370000.0,20.690001,20.360000,0.304385
16338,1951-01-04,20.870001,20.870001,20.870001,20.870001,3390000.0,20.870001,20.514000,0.204524
16337,1951-01-05,20.870001,20.870001,20.870001,20.870001,3390000.0,20.870001,20.628000,0.214057
16336,1951-01-08,21.000000,21.000000,21.000000,21.000000,2780000.0,21.000000,20.726001,0.181879
16335,1951-01-09,21.120001,21.120001,21.120001,21.120001,3800000.0,21.120001,20.840001,0.117047
16334,1951-01-10,20.850000,20.850000,20.850000,20.850000,3270000.0,20.850000,20.910001,0.161090
16333,1951-01-11,21.190001,21.190001,21.190001,21.190001,3490000.0,21.190001,20.942001,0.116060
16332,1951-01-12,21.110001,21.110001,21.110001,21.110001,2950000.0,21.110001,21.006001,0.149767
16331,1951-01-15,21.299999,21.299999,21.299999,21.299999,2830000.0,21.299999,21.054001,0.132778
16330,1951-01-16,21.459999,21.459999,21.459999,21.459999,3740000.0,21.459999,21.114000,0.165922


In [6]:
# First method: 30 day and 365 day averages

close_values = []
means = []
stds = []
indices = []

for index, row in df.iterrows():
    close_values.append(row['Close'])
    indices.append(index)
    if len(close_values) > 30:
        last_thirty = close_values[-31:-1]
        means.append(np.mean(last_thirty))
        stds.append(np.std(last_thirty, ddof=1))
    else:
        means.append(0)
        stds.append(0)

df['day_30'] = pd.Series(means, index=indices)
df['std_30'] = pd.Series(stds, index=indices)


close_values = []
means = []
stds = []
indices = []

for index, row in df.iterrows():
    close_values.append(row['Close'])
    indices.append(index)
    if len(close_values) > 365:
        last_365 = close_values[-366:-1]
        means.append(np.mean(last_365))
        stds.append(np.std(last_365, ddof=1))
    else:
        means.append(0)
        stds.append(0)

df['day_365'] = pd.Series(means, index=indices)
df['std_365'] = pd.Series(stds, index=indices)

In [7]:
# Add last two indicators

df['day_5/day_365'] = df['day_5'] / df['day_365']
df['std_5/std_365'] = df['std_5'] / df['std_365']

df = df.replace(np.inf, np.nan)

In [8]:
df.head(10)

,Date,Open,High,Low,Close,Volume,Adj Close,day_5,std_5,day_30,std_30,day_365,std_365,day_5/day_365,std_5/std_365
16589,1950-01-03,16.660000,16.660000,16.660000,16.660000,1260000.0,16.660000,0.000,0.000000,0.0,0.0,0.0,0.0,NaN,NaN
16588,1950-01-04,16.850000,16.850000,16.850000,16.850000,1890000.0,16.850000,0.000,0.000000,0.0,0.0,0.0,0.0,NaN,NaN
16587,1950-01-05,16.930000,16.930000,16.930000,16.930000,2550000.0,16.930000,0.000,0.000000,0.0,0.0,0.0,0.0,NaN,NaN
16586,1950-01-06,16.980000,16.980000,16.980000,16.980000,2010000.0,16.980000,0.000,0.000000,0.0,0.0,0.0,0.0,NaN,NaN
16585,1950-01-09,17.080000,17.080000,17.080000,17.080000,2520000.0,17.080000,0.000,0.000000,0.0,0.0,0.0,0.0,NaN,NaN
16584,1950-01-10,17.030001,17.030001,17.030001,17.030001,2160000.0,17.030001,16.900,0.157956,0.0,0.0,0.0,0.0,NaN,NaN
16583,1950-01-11,17.090000,17.090000,17.090000,17.090000,2630000.0,17.090000,16.974,0.089051,0.0,0.0,0.0,0.0,NaN,NaN
16582,1950-01-12,16.760000,16.760000,16.760000,16.760000,2970000.0,16.760000,17.022,0.067602,0.0,0.0,0.0,0.0,NaN,NaN
16581,1950-01-13,16.670000,16.670000,16.670000,16.670000,3330000.0,16.670000,16.988,0.134796,0.0,0.0,0.0,0.0,NaN,NaN
16580,1950-01-16,16.719999,16.719999,16.719999,16.719999,1460000.0,16.719999,16.926,0.196545,0.0,0.0,0.0,0.0,NaN,NaN


In [9]:
np.std(df['Close'].head(5))

0.14127986409959434

In [10]:
# Second method: Pandas rolling_mean and shift methods

df2 = pd.read_csv('sphist.csv')
df2['Date'] = pd.to_datetime(df2['Date'])
df2.sort_values(by='Date', ascending=True, inplace=True)

df2['day_5'] = df2['Close'].rolling(window=5, center=False).mean().shift(periods=1)
df2['std_5'] = df2['Close'].rolling(window=5, center=False).std().shift(periods=1)
df2['day_30'] = df2['Close'].rolling(window=30, center=False).mean().shift(periods=1)
df2['std_30'] = df2['Close'].rolling(window=30, center=False).std().shift(periods=1)
df2['day_365'] = df2['Close'].rolling(window=365, center=False).mean().shift(periods=1)
df2['std_365'] = df2['Close'].rolling(window=365, center=False).std().shift(periods=1)
df2['day_5/day_365'] = df2['day_5'] / df2['day_365']
df2['std_5/std_365'] = df2['std_5'] / df2['std_365']

df2 = df2.replace(np.inf, np.nan)

In [11]:
df2.head(10)

,Date,Open,High,Low,Close,Volume,Adj Close,day_5,std_5,day_30,std_30,day_365,std_365,day_5/day_365,std_5/std_365
16589,1950-01-03,16.660000,16.660000,16.660000,16.660000,1260000.0,16.660000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16588,1950-01-04,16.850000,16.850000,16.850000,16.850000,1890000.0,16.850000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16587,1950-01-05,16.930000,16.930000,16.930000,16.930000,2550000.0,16.930000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16586,1950-01-06,16.980000,16.980000,16.980000,16.980000,2010000.0,16.980000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16585,1950-01-09,17.080000,17.080000,17.080000,17.080000,2520000.0,17.080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16584,1950-01-10,17.030001,17.030001,17.030001,17.030001,2160000.0,17.030001,16.900,0.157956,NaN,NaN,NaN,NaN,NaN,NaN
16583,1950-01-11,17.090000,17.090000,17.090000,17.090000,2630000.0,17.090000,16.974,0.089051,NaN,NaN,NaN,NaN,NaN,NaN
16582,1950-01-12,16.760000,16.760000,16.760000,16.760000,2970000.0,16.760000,17.022,0.067602,NaN,NaN,NaN,NaN,NaN,NaN
16581,1950-01-13,16.670000,16.670000,16.670000,16.670000,3330000.0,16.670000,16.988,0.134796,NaN,NaN,NaN,NaN,NaN,NaN
16580,1950-01-16,16.719999,16.719999,16.719999,16.719999,1460000.0,16.719999,16.926,0.196545,NaN,NaN,NaN,NaN,NaN,NaN


Since you're computing indicators that use historical data, there are some rows where there isn't enough historical data to generate them. Some of the indicators use 365 days of historical data, and the dataset starts on 1950-01-03. Thus, any rows that fall before 1951-01-03 don't have enough historical data to compute all the indicators. You'll need to remove these rows before you split the data.

If you have a Dataframe df, you can select any rows with the Date column greater than 1951-01-02 using df[df["Date"] > datetime(year=1951, month=1, day=2)].

<b>Instructions</b><br>
- Remove any rows from the DataFrame that fall before 1951-01-03.
- Use the dropna method to remove any rows with NaN values. Pass in the axis=0 argument to drop rows.
- Generate two new dataframes to use in making our algorithm. train should contain any rows in the data with a date less than 2013-01-01. test should contain any rows with a date greater than or equal to 2013-01-01.

In [12]:
df2 = df2.dropna()

train = df2[df2['Date'] < datetime(year=2013, month=1, day=1)]
test = df2[df2['Date'] >= datetime(year=2013, month=1, day=1)]

In [13]:
train.head()

,Date,Open,High,Low,Close,Volume,Adj Close,day_5,std_5,day_30,std_30,day_365,std_365,day_5/day_365,std_5/std_365
16224,1951-06-19,22.020000,22.020000,22.020000,22.020000,1100000.0,22.020000,21.800,0.256223,21.703333,0.473595,19.447726,1.790253,1.120954,0.143121
16223,1951-06-20,21.910000,21.910000,21.910000,21.910000,1120000.0,21.910000,21.900,0.213659,21.683000,0.444648,19.462411,1.789307,1.125246,0.119409
16222,1951-06-21,21.780001,21.780001,21.780001,21.780001,1100000.0,21.780001,21.972,0.092574,21.659667,0.411452,19.476274,1.788613,1.128142,0.051758
16221,1951-06-22,21.549999,21.549999,21.549999,21.549999,1340000.0,21.549999,21.960,0.115108,21.631000,0.368514,19.489562,1.787659,1.126757,0.064390
16220,1951-06-25,21.290001,21.290001,21.290001,21.290001,2440000.0,21.290001,21.862,0.204132,21.599000,0.329130,19.502082,1.786038,1.121008,0.114293


In [14]:
test.head()

,Date,Open,High,Low,Close,Volume,Adj Close,day_5,std_5,day_30,std_30,day_365,std_365,day_5/day_365,std_5/std_365
738,2013-01-02,1426.189941,1462.430054,1426.189941,1462.420044,4.202600e+09,1462.420044,1418.641992,9.820801,1414.258667,17.834740,1327.534055,90.463948,1.068629,0.108560
737,2013-01-03,1462.420044,1465.469971,1455.530029,1459.369995,3.829730e+09,1459.369995,1425.793994,22.261321,1417.676668,16.852563,1327.908247,90.738976,1.073714,0.245334
736,2013-01-04,1459.369995,1467.939941,1458.989990,1466.469971,3.424290e+09,1466.469971,1433.702002,26.274326,1420.092668,17.470824,1328.224877,90.995857,1.079412,0.288742
735,2013-01-07,1466.469971,1466.469971,1456.619995,1461.890015,3.304970e+09,1461.890015,1443.376001,27.945242,1422.714665,18.339803,1328.557617,91.279049,1.086423,0.306152
734,2013-01-08,1461.890015,1461.890015,1451.640015,1457.150024,3.601600e+09,1457.150024,1455.267993,16.453319,1425.076664,18.678333,1328.898603,91.544368,1.095093,0.179731


Now, you can define an error metric, train a model using the train data, and make predictions on the test data.

It's recommended to use Mean Absolute Error, also called MAE, as an error metric, because it will show you how "close" you were to the price in intuitive terms. Mean Squared Error, or MSE, is an alternative that is more commonly used, but makes it harder to intuitively tell how far off you are from the true price because it squares the error.

<b>Instructions</b><br>
- Pick an error metric.
- Initialize an instance of the LinearRegression class.
- Train a linear regression model, using the train Dataframe. Leave out all of the original columns (Close, High, Low, Open, Volume, Adj Close, Date) when training your model. These all contain knowledge of the future that you don't want to feed the model. Use the Close column as the target.
- Make predictions for the Close column of the test data, using the same columns for training as you did with train.
- Compute the error between the predictions and the Close column of test.

In [15]:
lr = LinearRegression()
drop_cols = ['Close', 'High', 'Low', 'Open', 'Volume', 'Adj Close', 'Date']
keep_cols = train.columns.drop(drop_cols)

lr.fit(train[keep_cols], train['Close'])
predict_train = pd.Series(lr.predict(train[keep_cols]), index=train['Close'].index, name='Prediction')
predict_test = pd.Series(lr.predict(test[keep_cols]), index=test['Close'].index, name='Prediction')
mae_train = mean_absolute_error(train['Close'], predict_train)
mae_test = mean_absolute_error(test['Close'], predict_test)

predict_vs_actual = pd.concat([predict_test, test['Close']], axis=1)

print('Mean absolute error (train):', mae_train)
print('Mean absolute error (test):', mae_test)

Mean absolute error (train): 4.980320278598513
Mean absolute error (test): 16.216208263589802


/usr/local/lib/python3.6/site-packages/scipy/linalg/basic.py:1226: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


In [16]:
predict_vs_actual.head()

,Prediction,Close
738,1418.706799,1462.420044
737,1423.902102,1459.369995
736,1431.570688,1466.469971
735,1441.539769,1461.890015
734,1456.105008,1457.150024


Congratulations! You can now predict the S&P500 (with some error). You can improve the error of this model significantly, though. Think about some indicators that might be helpful to compute.

Here are some ideas that might be helpful:

- The average volume over the past five days.
- The average volume over the past year.
- The ratio between the average volume for the past five days, and the average volume for the past year.
- The standard deviation of the average volume over the past five days.
- The standard deviation of the average volume over the past year.
- The ratio between the standard deviation of the average volume for the past five days, and the standard deviation of the average volume for the past year.
- The year component of the date.
- The ratio between the lowest price in the past year and the current price.
- The ratio between the highest price in the past year and the current price.
- The year component of the date.
- The month component of the date.
- The day of week.
- The day component of the date.
- The number of holidays in the prior month.

<b>Instructions</b><br>
Add 2 additional indicators to your dataframe, and see if the error is reduced. You'll need to insert these indicators at the same point where you insert the others, before you clean out rows with NaN values and split the dataframe into train and `test.

In [19]:
# Add avg volume 5 days, avg vol 365 days, avg_vol_5/avg_vol_365, std_vol_5, std_vol_365, std_vol_5/std_vol_365,
# min_365/current, and max_365/current

df3 = pd.read_csv('sphist.csv')
df3['Date'] = pd.to_datetime(df3['Date'])
df3.sort_values(by='Date', ascending=True, inplace=True)

df3['close_mean_5'] = df3['Close'].rolling(window=5, center=False).mean().shift(periods=1)
df3['close_std_5'] = df3['Close'].rolling(window=5, center=False).std().shift(periods=1)
df3['close_mean_30'] = df3['Close'].rolling(window=30, center=False).mean().shift(periods=1)
df3['close_std_30'] = df3['Close'].rolling(window=30, center=False).std().shift(periods=1)
df3['close_mean_365'] = df3['Close'].rolling(window=365, center=False).mean().shift(periods=1)
df3['close_std_365'] = df3['Close'].rolling(window=365, center=False).std().shift(periods=1)

df3['vol_mean_5'] = df3['Volume'].rolling(window=5, center=False).mean().shift(periods=1)
df3['vol_mean_365'] = df3['Volume'].rolling(window=365, center=False).mean().shift(periods=1)
df3['vol_std_5'] = df3['Volume'].rolling(window=5, center=False).std().shift(periods=1)
df3['vol_std_365'] = df3['Volume'].rolling(window=365, center=False).std().shift(periods=1)

df3['min_365/current'] = (df3['Close'].rolling(window=365, center=False).min() / df3['Close']).shift(periods=1)
df3['max_365/current'] = (df3['Close'].rolling(window=365, center=False).max() / df3['Close']).shift(periods=1)

df3['close_mean_5/close_mean_365'] = df3['close_mean_5'] / df3['close_mean_365']
df3['close_std_5/close_std_365'] = df3['close_std_5'] / df3['close_std_365']
df3['vol_mean_5/vol_mean_365'] = df3['vol_mean_5'] / df3['vol_mean_365']
df3['vol_std_5/vol_std_365'] = df3['vol_mean_5'] / df3['vol_mean_365']

df3 = df3.replace(np.inf, np.nan)
df3 = df3.dropna()

train = df3[df3['Date'] < datetime(year=2013, month=1, day=1)]
test = df3[df3['Date'] >= datetime(year=2013, month=1, day=1)]

lr = LinearRegression()
drop_cols = ['Close', 'High', 'Low', 'Open', 'Volume', 'Adj Close', 'Date']
keep_cols = train.columns.drop(drop_cols)

lr.fit(train[keep_cols], train['Close'])
predict_train = pd.Series(lr.predict(train[keep_cols]), index=train['Close'].index, name='Prediction')
predict_test = pd.Series(lr.predict(test[keep_cols]), index=test['Close'].index, name='Prediction')
mae_train = mean_absolute_error(train['Close'], predict_train)
mae_test = mean_absolute_error(test['Close'], predict_test)

predict_vs_actual = pd.concat([predict_test, test['Close']], axis=1)

print('Mean absolute error (train):', mae_train)
print('Mean absolute error (test):', mae_test)

Mean absolute error (train): 5.328402572366055
Mean absolute error (test): 15.798940787588014


In [18]:
predict_vs_actual.head()

,Prediction,Close
738,1422.815487,1462.420044
737,1428.746153,1459.369995
736,1435.100279,1466.469971
735,1444.164678,1461.890015
734,1457.161269,1457.150024


There's a lot of improvement still to be made on the indicator side, and we urge you to think of better indicators that you could use for prediction. We can also make significant structural improvements to the algorithm, and pull in data from other sources.

Accuracy would improve greatly by making predictions only one day ahead. For example, train a model using data from 1951-01-03 to 2013-01-02, make predictions for 2013-01-03, and then train another model using data from 1951-01-03 to 2013-01-03, make predictions for 2013-01-04, and so on. This more closely simulates what you'd do if you were trading using the algorithm.

You can also improve the algorithm used significantly. Try other techniques, like a random forest, and see if they perform better.

You can also incorporate outside data, such as the weather in New York City (where most trading happens) the day before, and the amount of Twitter activity around certain stocks.

You can also make the system real-time by writing an automated script to download the latest data when the market closes, and make predictions for the next day.

Finally, you can make the system "higher-resolution". You're currently making daily predictions, but you could make hourly, minute-by-minute, or second by second predictions. This will require obtaining more data, though. You could also make predictions for individual stocks instead of the S&P500.

You can write scripts and explore here, or download the code to your computer using the download icon to the right. You'll then be able to run the scripts on your own computer.